#### @prompt example

In [1]:
from openbb import obb
import os
from magentic import prompt
from openbb import obb
obb.account.login(pat=os.environ['PAT_KEY'])
FMP_KEY = os.environ['FMP_KEY']


## Six Step Dance via Langchain
1. Taking into Account Company Industry (Growth / Value)
    1. Growth / Value
    2. Sector outperforming market or not
    3. Company outperforming sector
    4. PE vs Industry
2. Determining Company Financial Stability
    1. Debt To Equity
    2. Current Ratio
    3. Interest Margin Cover
3. Looking at Historical Earning and Growth
    1. Sales Growth
    2. EPS Growth
    3. Return on Invested Capital
    4. Profit Margin
    5. Cashflow Growth
    6. Equity Growth
    7. Return on Equity
4. Understanding Earnings and Sales Expectations
    1. Analyst Estimates
    2. PEG
5. Checking out competition
    1. P/E
    2. P/S
    3.MOAT
6. Estimating Company Values
    1. Insider Buying
    2. Senate TRading
    3. 13f

## Buffett Criteria
### Gross Margin Indicates profitability > 40% GrossProfit/Revenue GROSS PM close to 40 for recent years
### SG&A Margin efficiency in managing overhead costs < 30% SG&A Expenses / Gross Profit
### R&D Margin r&d spending relative to profitability < 30% R&D Expenses / Gross Profit
### Interest Expense Margin reliance on debt and financial health < 15 % Interest Expense / Operating Income
### Income Tax Margin Gauges tax efficiency ~ 20% Income Tax / Pre-Tax Income
### Profit Margin Shows overall profitabilty relative to revenue > 20% Net Income / Revenue
### EPS Growth Indicates growth and consistency in earnings >0 and growing Net Income / Shares Outstanding




### Attempting to write a Small Agent

In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [23]:
@tool
def get_performance(group:str) -> list:
    """ Return performance by sector or industyr industry performance for last week, last month, last quarter, last half year and last year"""
    return obb.equity.compare.groups(group=group, metric='performance', provider='finviz').to_llm()

@tool
def get_strong_buy_for_sector(sector : str) -> list :
    """ Return the strong buy recommendation for a given sector"""
    new_sector = '_'.join(sector.lower().split())
    data = obb.equity.screener(provider='finviz', sector=new_sector, recommendation='buy')
    return data.to_llm()

@tool
def get_best_stock_performers_for_sector(sector:str) -> list :
    """ Return the best  5 stock performers for last week and last month for a given sector"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Sector' : sector, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=5)
    return data.to_llm()

@tool
def get_best_stock_performers_for_industry(industry:str) -> list :
    """ Return the best  5 stock performers for last week and last month for an industry"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Sector' : industry, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=10)
    return data.to_llm()

@tool
def get_valuation_for_sectors(input:str) -> list:
    """ Return valuation metrics for the sector provided as input"""
    data = obb.equity.compare.groups(group='sector', metric='valuation', provider='finviz').to_df()
    
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )

@tool
def get_valuation_for_industries(input:str) -> list:
    """ Return valuation meetrics for the industry provided as input"""
    data =  obb.equity.compare.groups(group='industry', metric='valuation', provider='finviz').to_df()
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )


@tool
def get_valuation_for_company(ticker:str) -> list:
    """ Return Valuation metrics for a company"""
    # requires obb login
    obb.account.login(pat=os.environ['PAT_KEY'])
    return obb.equity.fundamental.ratios_ttm(symbol='AAPL', provider='fmp', limit=1).to_llm()

#get_strong_buy_for_sector.invoke('Consumer Cyclical')
get_valuation_for_sectors.invoke('Utilities')

'[{"name":"Utilities","market_cap":1543480000000,"performance_1d":-0.056,"pe":19.2,"forward_pe":15.32,"peg":2.42,"eps_growth_past_5y":0.0529,"eps_growth_next_5y":0.0795,"volume":394950000,"price_to_sales":2.15,"price_to_book":2.02,"price_to_cash":21.41,"price_to_free_cash_flow":72.31,"sales_growth_past_5_years":0.0601}]'

In [24]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful stock financial analys, but don't know current events. Use the toools as much as you can to retrieve informations",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [25]:
tools = [get_performance, get_strong_buy_for_sector, get_best_stock_performers_for_sector, get_valuation_for_sectors, get_valuation_for_industries,get_valuation_for_company]
llm_with_tools = llm.bind_tools(tools)

In [26]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

### Chat Memory

In [27]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationTokenBufferMemory
#from langchain_core.messages import count_tokens_approximately
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain.agents import AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage




MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful stock recommendation assistant , but dont know current events so you should use your tools as much as you can.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [28]:
chat_history = []
chat_history.append(HumanMessage(content="Your question here"))
chat_history.append(AIMessage(content="AI response here"))
memory = ConversationTokenBufferMemory(
    llm=llm,  # Required for token counting
    max_token_limit=14000,  # Leave buffer for functions + responses
    memory_key="chat_history",  # Must match your prompt's key
    return_messages=True
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: memory.load_memory_variables(x)["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [31]:
input1 = "Find the sector that has shown a constant postive performance across quarter, month and week.Once you get it, please extract its valuation metrics and the top performers for this sector."
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_performance` with `{'group': 'sector'}`


[{"name":"Energy","performance_1d":-0.0861,"performance_1w":-0.1313,"performance_1m":-0.0623,"performance_3m":-0.0855,"performance_6m":-0.1169,"performance_1y":-0.164,"performance_ytd":-0.0726,"analyst_recommendation":1.89,"volume":1410000000,"volume_average":701420000,"volume_relative":2.01},{"name":"Technology","performance_1d":-0.0634,"performance_1w":-0.114,"performance_1m":-0.1529,"performance_3m":-0.221,"performance_6m":-0.1597,"performance_1y":-0.0643,"performance_ytd":-0.22,"analyst_recommendation":1.74,"volume":4059999999,"volume_average":2810000000,"volume_relative":1.45},{"name":"Financial","performance_1d":-0.0698,"performance_1w":-0.1029,"performance_1m":-0.1021,"performance_3m":-0.0786,"performance_6m":-0.0238,"performance_1y":0.0628,"performance_ytd":-0.0809,"analyst_recommendation":2.06,"volume":2250000000,"volume_average":1310000000,"volume_relative":1.72},{"name":"Basic M

In [30]:
input1 = "Now find me the best strong buy companies for this sector"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `get_strong_buy_for_sector` with `{'sector': 'Technology'}`


[{"symbol":"NVTS","name":"Navitas Semiconductor Corp","country":"USA","sector":"Technology","industry":"Semiconductors","market_cap":344860000.0,"price":1.81,"change_percent":0.1277,"volume":6657300,"price_to_earnings":null},{"symbol":"GAUZ","name":"Gauzy Ltd","country":"Israel","sector":"Technology","industry":"Electronic Components","market_cap":157060000.0,"price":8.39,"change_percent":0.0756,"volume":64820,"price_to_earnings":null},{"symbol":"RUN","name":"Sunrun Inc","country":"USA","sector":"Technology","industry":"Solar","market_cap":1580000000.0,"price":6.99,"change_percent":0.0607,"volume":16146299,"price_to_earnings":null},{"symbol":"AEHR","name":"Aehr Test Systems","country":"USA","sector":"Technology","industry":"Semiconductor Equipment & Materials","market_cap":215710000.0,"price":7.26,"change_percent":0.0606,"volume":1229062,"price_to_earnings":9.72},{"symbol":"

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 18149 tokens (17940 in the messages, 209 in the functions). Please reduce the length of the messages or functions.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "Which stock are recommented for this sector best performing sector", "chat_history": chat_history})